# Fitting

In [ ]:
import sys
sys.path.insert(0, '../../src/')

import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import tensorflow as tf

from qiskit.quantum_info import Operator
from tqdm.notebook import tqdm

from kraus_channels import KrausMap, isomery_to_kraus
from loss_functions import ProbabilityMSE, ProbabilityRValue
from optimization import ModelSPAM, ModelQuantumMap, Logger, model_saver
from quantum_channel import channel_fidelity
from experimental import counts_to_probs, generate_pauliInput_circuits, generate_pauli_circuits, marginalize_counts
from spam import SPAM, InitialState, CorruptionMatrix
from utils import saver
from quantum_circuits import pqc_basic
from spectrum import channel_spectrum, complex_spacing_ratio
from quantum_circuits import integrable_circuit


#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [6]:
def integrable_circuit(n, L):
    theta_list = [np.random.uniform(-np.pi, np.pi, n) for i in range(L)]
    sqrt_iSWAP = iSwapGate().power(1 / 2)

    circuit = qk.QuantumCircuit(n)
    for i, theta in enumerate(theta_list):

        offset = i%2
        for j in range(n):
            circuit.rz(theta[j], j)

        for j in range((n - offset) // 2):
            circuit.append(sqrt_iSWAP, [2 * j + offset, 2 * j + 1 + offset])


    return circuit


def nonintegrable_circuit(n, L):
    theta_list = [np.random.uniform(-np.pi, np.pi, 2*n) for i in range(L)]
    sqrt_iSWAP = iSwapGate().power(1 / 2)

    circuit = qk.QuantumCircuit(n)
    for i, theta in enumerate(theta_list):

        offset = i%2
        for j in range(n):
            circuit.ry(theta[j], j)
            circuit.rz(theta[j+n], j)

        for j in range((n - offset) // 2):
            circuit.append(sqrt_iSWAP, [2 * j + offset, 2 * j + 1 + offset])


    return circuit

In [11]:
def load_data(filename, n, seed, L):
    with open(filename, 'rb') as f:
        data = pkl.load(f)


    data = marginalize_counts(data, 0)

    targets = counts_to_probs(data)
    targets_spam = targets[:6**n]
    targets_map = targets[6**n:]

    np.random.seed(seed)
    
    circuit_target = integrable_circuit(n+1, L)
    unitary = Operator(circuit_target).data

    inputs_spam, _ = generate_pauliInput_circuits(n)
        
    inputs_map, circuit_list_map = (
                generate_pauli_circuits(n, None, N=5000-6**n)
        )
    
    return inputs_spam, targets_spam, inputs_map, targets_map, unitary

def fit_spam(inputs, 
             targets,
             num_iter = 1000,
             verbose = False):
    d = targets.shape[1]
    spam_model = SPAM(init = InitialState(d),
                    povm = CorruptionMatrix(d),
                    )

    spam_opt = ModelSPAM(spam_model, tf.keras.optimizers.Adam(learning_rate=0.01))
        
    spam_opt.pretrain(100, verbose=False)

    spam_opt.train(inputs = inputs,
                    targets = targets,
                    num_iter = num_iter,
                    verbose = verbose,
                )
    
    return spam_model
    

def fit_model(inputs, 
              targets, 
              spam_model,
              num_iter = 1000,
              verbose=False):
    d = targets.shape[1]
    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                        rank = d**2,
                                        spam = spam_model,
                                        ),
                    loss_function = ProbabilityMSE(),
                    optimizer = tf.optimizers.Adam(learning_rate=0.01),
                    logger = Logger(loss_function_list = [ProbabilityRValue()], sample_freq=100),
                )

    model.train(inputs = inputs,
                targets = targets,
                inputs_val = [inputs],
                targets_val = [targets],
                num_iter = num_iter,
                N = 500,
                verbose=verbose
                )
    
    return model

In [ ]:
path = 'data/chaos_exp_reorder/5q/integrable_L=10/'
n = 4
d = 2**n
L = 5

spam_list = []
model_list = []

for i in tqdm(range(10)):
    seed = 42 + i
    print(seed)
    inputs_spam, targets_spam, inputs_map, targets_map, unitary = load_data(path + f'seed_{seed}.pkl', n, seed, L)
    spam_model = fit_spam(inputs_spam, targets_spam, verbose=True)
    spam_list.append(spam_model) 

    tf.random.set_seed(50)
    model = fit_model(inputs_map, 
                     targets_map, 
                     spam_model, 
                     verbose=True)
    model_list.append(model)

model_saver(spam_list, f'../models/integrable_spam_{n}_L=10.model')
model_saver(model_list, f'../models/integrable_model_{n}_L=10.model')

  0%|          | 0/1 [00:00<?, ?it/s]

43


  0%|          | 0/1000 [00:00<?, ?it/s]

0.07547598889666043 
0.0036227812437261368 
0.00017837985156437874 
7.540714971109532e-05 
7.4082413028364e-05 
7.317879706849127e-05 
7.250800761923568e-05 
7.199874449480174e-05 
7.161272904363952e-05 
7.134468484672496e-05 


  0%|          | 0/1000 [00:00<?, ?it/s]

[0.0010737345912030793]
[0.5835856663488728]
[0.9147918937682042]
[0.9794836597015406]
[0.9918132217827754]
[0.9954883594355474]
[0.9967443321007059]
[0.9972572366228032]
[0.9975137352303175]
[0.9976674483411654]
[0.9977658910742685]
